In [15]:
import os, sys
from django.apps import apps as django_apps
from django.db.models.signals import post_delete
from datetime import datetime
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [16]:
post_delete.disconnect(dispatch_uid='offschedule_model_on_post_delete')

def delete_offschedule_instance(onsch_model_cls, subject_identifier, child_identifier):
    onsch = onsch_model_cls.objects.get(subject_identifier=subject_identifier,
                                        child_subject_identifier=child_identifier)
    offsch = CaregiverOffSchedule.objects.get(subject_identifier=onsch.subject_identifier,
                                              schedule_name=onsch.schedule_name)
    offsch.delete()
    try:
        ssh = SubjectScheduleHistory.objects.get(subject_identifier=onsch.subject_identifier,
                                                 schedule_name=onsch.schedule_name)
        ssh.schedule_status = 'onschedule'
        ssh.offschedule_datetime = None
        ssh.save()
    except SubjectScheduleHistory.DoesNotExist:
        print(subject_identifier, child_identifier, onsch.schedule_name)

    if 'quart' in onsch.schedule_name or 'qt' in onsch.schedule_name:
        _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
            onschedule_model=ssh.onschedule_model, name=ssh.schedule_name)
        schedule.put_on_schedule(subject_identifier=onsch.subject_identifier,
                                 onschedule_datetime=onsch.onschedule_datetime,
                                 schedule_name=onsch.schedule_name, )

In [17]:
from flourish_caregiver.models import CaregiverOffSchedule
from flourish_prn.models import CaregiverOffStudy

subject_identifier = 'B142-040990255-7'
child_identifier_on = 'B142-040990255-7-10'
child_identifier_off = 'B142-040990255-7-60'

In [5]:
offstudy = CaregiverOffStudy.objects.get(subject_identifier=subject_identifier)
offstudy.delete()

visit = MaternalVisit.objects.get(subject_identifier=subject_identifier,
                                  visit_code='1000M',
                                  visit_code_sequence=1)
visit.study_status = 'on study'
visit.save()

In [18]:
models = [OnScheduleCohortBEnrollment, OnScheduleCohortCFUSeq, OnScheduleCohortCFUQuarterly]

for model_cls in models:
    delete_offschedule_instance(model_cls, subject_identifier, child_identifier_on)

In [19]:
delete_offschedule_instance(OnScheduleCohortAAntenatal, subject_identifier, child_identifier_off)